In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import gcsfs

In [2]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
inexus = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-5planmemory-20221205/inexus/sfbay_baseline_default-1.0_2010__20221119.csv.gz', compression = 'gzip')

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_868\4075349023.py:1: DtypeWarning: Columns (27,84,88,157) have mixed types. Specify dtype option on import or set low_memory=False.
  inexus = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-5planmemory-20221205/inexus/sfbay_baseline_default-1.0_2010__20221119.csv.gz', compression = 'gzip')


In [7]:
inexus.head()

,Unnamed: 0,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,BlockGroupEnd,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,0,4.0,1513.0,59431.0,57128.0,2303.0,6.607127,othdiscr,Home,0.0,2303.0,0.0,0.0,60240.360,0.0,0.000,0.0,0.0,60240.360,0.0,"body-4,358558,358558,body-4",car,car,"358558, body-4","1.0, 0.0, 0.0, 1.0",60240.360,0,NaN,"Residential, Public",0,0,0,0,0,0,0.000,0.0,0.0,0.000000e+00,9.943529e+07,9.943529e+07,9.943529e+07,1005.0,401.0,0.0,0.0,0.000000,0.007350,0.0,0.0,2303.0,0.0,Home_to_othdiscr,car,car,1513.0,4.0,865.0,189.0,othdiscr,1.0,True,1.0,othdiscr,401.0,1

In [17]:
events_2010_3_noPlan = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/beam/year-2010-iteration-3/ITERS/it.0/0.events.csv.gz', compression = 'gzip')

KeyboardInterrupt: 

In [16]:
final_trips_2010_3_noPlan = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/activitysim/year-2010-iteration-3/final_trips.csv.gz', compression = 'gzip')

In [4]:
plans_2010_3 = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-20221122/activitysim/year-2010-iteration-3/plans.csv.gz', compression = 'gzip')

In [14]:
inexus = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-20221122/inexus/sfbay_baseline_default-1.0_2010__20221119.csv.gz', compression = 'gzip')

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_556\3162206917.py:1: DtypeWarning: Columns (27,84,88,157) have mixed types. Specify dtype option on import or set low_memory=False.
  inexus = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-20221122/inexus/sfbay_baseline_default-1.0_2010__20221119.csv.gz', compression = 'gzip')


In [10]:
# Mapping BEAM and ASIM modes
conditions  = [(inexus['mode_choice_actual_BEAM'] == 'walk')&(inexus['trip_mode_AS_trips'] == 'WALK'),
               (inexus['mode_choice_actual_BEAM'] == 'bike')&(inexus['trip_mode_AS_trips'] == 'BIKE'),
               (inexus['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(inexus['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (inexus['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(inexus['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (inexus['mode_choice_actual_BEAM'] == 'car_hov3')&(inexus['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (inexus['mode_choice_actual_BEAM'] == 'car_hov3')&(inexus['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (inexus['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(inexus['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (inexus['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(inexus['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (inexus['mode_choice_actual_BEAM'] == 'car_hov2')&(inexus['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (inexus['mode_choice_actual_BEAM'] == 'car_hov2')&(inexus['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (inexus['mode_choice_actual_BEAM'] == 'car')&(inexus['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (inexus['mode_choice_actual_BEAM'] == 'car')&(inexus['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (inexus['mode_choice_actual_BEAM'] == 'walk_transit')&(inexus['trip_mode_AS_trips'] == 'WALK_LRF'),
               (inexus['mode_choice_actual_BEAM'] == 'walk_transit')&(inexus['trip_mode_AS_trips'] == 'WALK_LOC'),
               (inexus['mode_choice_actual_BEAM'] == 'walk_transit')&(inexus['trip_mode_AS_trips'] == 'WALK_EXP'),
               (inexus['mode_choice_actual_BEAM'] == 'walk_transit')&(inexus['trip_mode_AS_trips'] == 'WALK_HVY'),
               (inexus['mode_choice_actual_BEAM'] == 'ride_hail')&(inexus['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (inexus['mode_choice_actual_BEAM'] == 'ride_hail')&(inexus['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (inexus['mode_choice_actual_BEAM'] == 'ride_hail')&(inexus['trip_mode_AS_trips'] == 'TAXI'),
               (inexus['mode_choice_actual_BEAM'] == 'walk_transit')&(inexus['trip_mode_AS_trips'] == 'WALK_COM'),
               (inexus['mode_choice_actual_BEAM'] == 'drive_transit')&(inexus['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (inexus['mode_choice_actual_BEAM'] == 'drive_transit')&(inexus['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (inexus['mode_choice_actual_BEAM'] == 'drive_transit')&(inexus['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (inexus['mode_choice_actual_BEAM'] == 'drive_transit')&(inexus['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (inexus['mode_choice_actual_BEAM'] == 'drive_transit')&(inexus['trip_mode_AS_trips'] == 'DRIVE_HVY')]


choices = [inexus['WALK'], inexus['BIKE'], inexus['SHARED3FREE'], inexus['SHARED3PAY'],
           inexus['SHARED3FREE'], inexus['SHARED3PAY'], inexus['SHARED2FREE'], 
           inexus['SHARED2PAY'], inexus['SHARED2PAY'], inexus['SHARED2FREE'], 
           inexus['DRIVEALONEFREE'], inexus['DRIVEALONEPAY'], inexus['WALK_LRF'], 
           inexus['WALK_LOC'], inexus['WALK_EXP'], inexus['WALK_HVY'], inexus['TNC_SINGLE'], 
           inexus['TNC_SHARED'], inexus['TAXI'], inexus['WALK_COM'], 
           inexus['DRIVE_COM'], inexus['DRIVE_LRF'], inexus['DRIVE_LOC'], inexus['DRIVE_EXP'],
           inexus['DRIVE_HVY']]

In [11]:
inexus['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)

In [30]:
inexus[(inexus['IDMerged']==167)&(inexus['tour_id']==6875)].head(10)

,Unnamed: 0,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,BlockGroupEnd,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
54,54,167.000000,55001.000000,22713.000000,21920.000000,793.000000,2.445813,escort,Home,0.000000,793.000000,0.000000,0.000000,22299.654000,0.000000,0.000000,0.000000,0.000000,22299.654000,0.000000,"body-167,296415,296415,body-167",car,car,"296415, body-167","1.0, 0.0, 0.0, 1.0",22299.654000,0,NaN,"Residential, Public",0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,38644532.085171,38644532.085171,38644532.085171,1005.000000,862.000000,0.0

In [ ]:
inexus[(inexus['IDMerged']==167)&(inexus['tour_id']==6875)].head(10)

In [39]:
inexus[(inexus['IDMerged']==4370)&(inexus['tour_id']==179176)].head()

,Unnamed: 0,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,BlockGroupEnd,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
1762,1762,4370.000000,1433409.000000,66203.000000,65804.000000,399.000000,0.000000,shopping,Home,0.000000,0.000000,399.000000,0.000000,1994.629000,0.000000,0.000000,1994.629000,0.000000,0.000000,0.000000,"body-4370,748939,body-4370",bike,bike,"748939, body-4370","1.0, 1.0, 1.0",1994.629000,0,NaN,NaN,0,0,0,0,0,0,21143.067400,0.000000,0.000000,0.000000,0.000000,0.000000,21143.067400,999.000000,1004.000000,0.000000,0.000000,0.000000,0.000000

In [42]:
inexus[(inexus['IDMerged']==125)].head()

,Unnamed: 0,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,BlockGroupEnd,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
24,24,125.000000,38366.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,NaN,NaN,51823,NaN,0.000000,0,NaN,Residential,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_to_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [15]:
inexus[(inexus['has_school_kid'].isna())].shape

(957135, 253)

In [13]:
inexus[(inexus['Realized_INEXUS'].isna())].tail()

,Unnamed: 0,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,BlockGroupEnd,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
983596,983596,6808769.0,2.233276e+09,99778.0,96432.0,3346.0,0.000000,socialHome,Homesocial,0.0,3346.0,0.0,0.0,78978.075000,0.0,0.000000,0.0,0.0,78978.075,0.0,"body-6808769,teleportationSharedVehicle-1-6808...",hov3_teleportation,hov3_teleportation,NaN,NaN,78978.075000,0,NaN,NaN,0,0,0,0,0,0,0.000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1403.0,1403.0,0.0,0.0,0.0,0.000000,0.0,0.0,3346.0,0.0,Homesocial_to_socialHome,car,car,NaN,NaN

In [57]:
plans_2010_3.head()

,trip_id,person_id,number_of_participants,trip_mode,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time
0,1513.000000,4.000000,1.000000,DRIVEALONEFREE,2,leg,NaN,NaN,NaN,NaN
1,1517.000000,4.000000,1.000000,DRIVEALONEPAY,4,leg,NaN,NaN,NaN,NaN
2,1518.000000,4.000000,1.000000,DRIVEALONEFREE,6,leg,NaN,NaN,NaN,NaN
3,2889.000000,8.000000,1.000000,DRIVEALONEFREE,2,leg,NaN,NaN,NaN,NaN
4,2893.000000,8.000000,1.000000,DRIVEALONEFREE,4,leg,NaN,NaN,NaN,NaN


In [16]:
# Merge BEAM households and persons 
final_trips_2010_3 = final_trips_2010_3.sort_values(by=['trip_id']).reset_index(drop=True)
plans_2010_3 = plans_2010_3.sort_values(by=['trip_id']).reset_index(drop=True)
tripPlan = pd.merge(left=final_trips_2010_3, right=plans_2010_3, how='left', on='trip_id')
planTrip = pd.merge(left=plans_2010_3, right=final_trips_2010_3, how='left', on='trip_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [32]:
planTrip[planTrip['mode_choice_logsum'].isna()].head()

,trip_id,person_id_x,number_of_participants,trip_mode_x,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time,person_id_y,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode_y,mode_choice_logsum
2459611,NaN,4.0,NaN,NaN,1,activity,Home,-122.227937,37.854097,15.567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459612,NaN,4.0,NaN,NaN,3,activity,othdiscr,-122.117242,37.457796,16.624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459613,NaN,4.0,NaN,NaN,5,activity,social,-122.271282,37.835106,16.828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459614,NaN,4.0,NaN,NaN,7,activity,Home,-122.227937,37.854097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459615,NaN,8.0,NaN,NaN,1,activity,Home,-122.227937,37.854097,16.099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
np.where(tripPlan['trip_mode_x'] != tripPlan['trip_mode_y']) 

(array([], dtype=int64),)

In [59]:
plans_2010_3[plans_2010_3['trip_id'] == 37970]

,trip_id,person_id,number_of_participants,trip_mode,PlanElementIndex,ActivityElement,ActivityType,x,y,departure_time
15,37970.000000,115.000000,1.000000,SHARED2FREE,4,leg,NaN,NaN,NaN,NaN


In [64]:
plans_2010_3.trip_id.nunique()

2459611

In [58]:
final_trips_2010_3[final_trips_2010_3['trip_id'] == 37970]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum
15,37970,115,1114,4746,school,2,True,2,school,527,626,NaN,7.000000,SHARED2FREE,-2.705016


In [82]:
final_trips_2010_3[final_trips_2010_3['trip_id'].isna()].head()

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum


In [65]:
df=pd.concat([final_trips_2010_3, plans_2010_3],axis=1, keys = ['final_trips_2010_3', 'plans_2010_3'])

In [74]:
df['trip_id_diff'] = np.where(df['final_trips_2010_3']['trip_id']==df['plans_2010_3']['trip_id'], 0,1)

In [18]:
%%time
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
eventsSF = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/beam/year-2010-iteration-3/ITERS/it.0/0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:22: DtypeWarning: Columns (8,14,22,25,37,42,43,48,49,55) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 5min 32s
Wall time: 6min 28s


#### Read the files directly from S3

In [4]:
%%time
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
eventsSF = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-20221122/beam/year-2015-iteration-3/ITERS/it.0/0.events.csv.gz', compression = 'gzip', dtype = dtypes)

KeyboardInterrupt: 

In [19]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

In [20]:
# Adding scenario info
eventsSF['scenario'] = "baseline"
eventsSF['scenario'] = eventsSF['scenario'].astype("category")
eventsSF['lever'] = "default"
eventsSF['lever'] = eventsSF['lever'].astype("category")
eventsSF['year'] = 2010
eventsSF['lever_position'] = 1

In [21]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [22]:
eventsSF = eventsSF.copy()

In [23]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

#### New

In [24]:
eventsSF = eventsSF[~((eventsSF.type.str.contains("ParkingEvent", na=False))&(eventsSF['time']==0))].reset_index(drop=True)

In [25]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [26]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [27]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [28]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [29]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [30]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

CPU times: total: 1min 42s
Wall time: 1min 42s


In [31]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [32]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [33]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [34]:
#IDnan = eventsSF[eventsSF['IDMerged'].isna()]
#IDnan.shape

In [35]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

CPU times: total: 4min 30s
Wall time: 4min 30s


In [36]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [37]:
#IDnan = eventsSF[eventsSF['IDMerged'].isna()]
#IDnan.shape

In [38]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [39]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

CPU times: total: 17.3 s
Wall time: 17.3 s


In [40]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [41]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [42]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [43]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [44]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [45]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [46]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [47]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [48]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [49]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding the census blocks

In [ ]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [ ]:
# census_2010 = "https://beam-core-act.s3.amazonaws.com/deepDive/RawData/Census_Blocks/Scenario_2010_shp/"
BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [ ]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'BlockGroupStart')

In [38]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'BlockGroupEnd')

Exception ignored in: <function BaseGeometry.__del__ at 0x0000011FE360F280>
Traceback (most recent call last):
  File "C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\shapely\geometry\base.py", line 242, in __del__
    self.empty(val=None)
  File "C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\shapely\geometry\base.py", line 227, in empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 
Exception ignored in: <function BaseGeometry.__del__ at 0x0000011FE360F280>
Traceback (most recent call last):
  File "C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\shapely\geometry\base.py", line 242, in __del__
    self.empty(val=None)
  File "C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\shapely\geometry\base.py", line 227, in empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 
Exception ignored in: <function BaseGeometry.__del__ at 0x0000011FE360F280>
Traceback (most recent call last):
  File "C:\Users\nazanin\.conda\envs\geo_

KeyboardInterrupt: 

#### Adding new columns

In [50]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [51]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [52]:
eventsSF['duration_travelling'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [53]:
eventsSF['distance_travelling'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [54]:
eventsSF['distance_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [55]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['duration_travelling'], np.nan)

In [56]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['distance_travelling'], np.nan)

In [57]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['duration_travelling'], np.nan)

In [58]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['distance_travelling'], np.nan)

In [59]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['duration_travelling'], np.nan)

In [60]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [61]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['duration_travelling'], np.nan)

In [62]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['distance_travelling'], np.nan)

In [63]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['duration_travelling'], np.nan)

In [64]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [65]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [66]:
#eventsSF = eventsSF.set_index('IDMerged')

In [67]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [68]:
eventsSF['emissionFood'] = eventsSF['fuelFood'] * 8.3141841e-9 * 0

In [69]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [70]:
eventsSF['emissionElectricity'] = eventsSF['fuelElectricity'] * 2.77778e-10 * 947.2 * 0.0005

In [71]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [72]:
eventsSF['emissionDiesel'] = eventsSF['fuelDiesel'] * 8.3141841e-9 * 10.180e-3

In [73]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [74]:
eventsSF['emissionBiodiesel'] = eventsSF['fuelBiodiesel'] * 8.3141841e-9 * 10.180e-3

In [75]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [76]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [77]:
eventsSF['emissionGasoline'] = eventsSF['fuelGasoline'] * 8.3141841e-9 * 8.89e-3

In [78]:
# Marginal fuel
conditions  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled'), 
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),
               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')]
choices = [eventsSF['fuel_not_Food']/eventsSF['numPassengers'], 0 , eventsSF['fuelFood'], eventsSF['fuel_not_Food']]

In [79]:
eventsSF['fuel_marginal'] = np.select(conditions, choices, default=np.nan)

In [80]:
# Marginal emission
conditions1  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0), 
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelGasoline'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelBiodiesel'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelDiesel'].notna() != 0),             
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),

               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelElectricity'].notna() != 0),
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelGasoline'].notna() != 0),           
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelBiodiesel'].notna() != 0),   
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelDiesel'].notna() != 0),

               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelFood'].notna() != 0)]

choices1 = [eventsSF['emissionElectricity']/eventsSF['numPassengers'],
           eventsSF['emissionGasoline']/eventsSF['numPassengers'],
           eventsSF['emissionBiodiesel']/eventsSF['numPassengers'],
           eventsSF['emissionDiesel']/eventsSF['numPassengers'],           
           0 , 
           eventsSF['emissionFood'], 
           eventsSF['emissionElectricity'],
           eventsSF['emissionGasoline'],
           eventsSF['emissionBiodiesel'],
           eventsSF['emissionDiesel'],
           eventsSF['emissionFood']]

In [81]:
eventsSF['emission_marginal'] = np.select(conditions1, choices1, default=np.nan)

In [82]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [83]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [84]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Mode Choice planned and actual

In [85]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

CPU times: total: 1min 24s
Wall time: 1min 24s


In [86]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

CPU times: total: 1min 23s
Wall time: 1min 23s


In [87]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [88]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [89]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [90]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [94]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

CPU times: total: 1.64 s
Wall time: 1.64 s


In [95]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [96]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [98]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 
            'actEndTime': np.sum, 
            'duration_travelling': np.sum, 
            'cost_BEAM': np.sum, 
            'actStartType': np.sum, 
            'actEndType': np.sum, 
            'duration_walking': np.sum, 
            'duration_in_privateCar': np.sum, 
            'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 
            'distance_travelling': np.sum, 
            'duration_in_transit': np.sum, 
            'distance_walking': np.sum, 
            'distance_bike': np.sum, 
            'distance_ridehail': np.sum, 
            'distance_privateCar': np.sum, 
            'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'parkingType': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 
            'transit_subway': np.sum, 
            'transit_tram': np.sum, 
            'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 
            'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'lever_position': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str))),
            'fuelFood': np.sum, 
            'fuelElectricity': np.sum, 
            'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 
            'fuel_not_Food': np.sum, 
            'fuelGasoline': np.sum, 
            'fuel_marginal': np.sum,
            'emissionFood': np.sum, 
            'emissionElectricity': np.sum, 
            'emissionDiesel': np.sum, 
            'emissionGasoline': np.sum,
            'emissionBiodiesel': np.sum, 
            'emission_marginal': np.sum,
            'lever': lambda x: ', '.join(set(x.dropna().astype(str)))
           }).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

CPU times: total: 48min 16s
Wall time: 48min 53s


In [99]:
Person_Trip_eventsSF['duration_door_to_door'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [100]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['duration_door_to_door'] - Person_Trip_eventsSF['duration_travelling'] 

In [103]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [104]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"vehicleIds_estimate"}, inplace=True) 

In [105]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [123]:
Person_Trip_eventsSF.head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4
0,4,1513.000000,57513.000000,Home,61414.000000,othdiscr,6.609155,0.000000,60258.855000,62966.636000,0.000000,0.000000,64733.977000,1767.341000,2542.000000,0.000000,0.000000,0.000000,3901.000000,1359.000000,0.000000,0.000000,0.000000,0.000000,0.007940,0.000000,0.000000,0.000000,0.000000,93669.073000,107418015.519458,107511684.592458,107418015.519458,"body-4,358558,358558,body-4",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Residential, Residential, Public",,0,0,baseline,0,0,0,0,0,"358558, body-4",2010,3901.000000,0.000000,Home_to_othdiscr,car,car,car
1,4,1517.000000,61564.000000,othdiscr,67049.000000,social,6.618786,0.000000,63950.718000,60970.506000,0.000000,0.000000,65066.725000,4096.219000,2334.000000,0.000000,0.000000,0.000000,5485.000000,3151.000000,0.000000,0.000000,0.000000,0.000000,0.007577,0.000000,0.000000,0.000000,0.000000,217099.607000,102507259.727169,102724359.334169,102507259.727169,"body-4,358558,358558,body-4",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Public, Public",,0,0,baseline,0,0,0,0,0,"358558, body-4",2010,5485.000000,0.000000,othdiscr_to_social,car,car,car
2,4,1518.000000,67199.000000,social,68335.000000,Home,0.679361,0.000000,7352.261000,6194.061000,0.000000,0.000000,7352.261000,1158.200000,245.000000,0.000000,0.000000,0.000000,1136.000000,891.000000,0.000000,0.000000,0.000000,0.000000,0.000948,0.000000,0.000000,0.000000,0.000000,61384.600000,12822557.735979,12883942.335979,12822557.735979,"body-4,358558,358558,body-4",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Residential",,0,0,baseline,0,0,0,0,0,"358558, body-4",2010,1136.000000,0.000000,social_to_Home,car,car,car
3,8,2889.000000,58970.000000,Home,59781.000000,shopping,0.527114,0.000000,20273.607000,20273.607000,0.000000,0.000000,20273.607000,0.000000,811.000000,0.000000,0.000000,0.000000,811.000000,0.000000,0.000000,0.002237,0.000000,0.000000,0.000000,0.000000,0.000000,26435546.325127,0.000000,0.000000,0.000000,26435546.325127,26435546.325127,"body-8,8-emergency-0,8-emergency-0,body-8",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Public",,0,0,baseline,0,0,0,0,0,"body-8, 8-emergency-0",2010,811.000000,0.000000,Home_to_shopping,car,car,car
4,8,2893.000000,60267.000000,shopping,61172.000000,Home,0.508436,0.000000,19727.073000,19555.240000,0.000000,0.000000,19727.073000,171.833000,773.000000,0.000000,0.000000,0.000000,905.000000,132.000000,0.000000,0.002160,0.000000,0.000000,0.000000,0.000000,0.000000,25524040.238863,0.000000,9107.149000,0.000000,25533147.387863,25524040.238863,"body-8,8-emergency-0,8-emergency-0,body-8",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Residential",,0,0,baseline,0,0,0,0,0,"body-8, 8-emergency-0",2010,905.000000,0.000000,shopping_to_Home,car,car,car


In [107]:
final_trips_2010_3_noPlan.head()

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum
0,862905,2630,646,107863,shopping,1,True,1,shopping,1122,1005,NaN,13.000000,SHARED3FREE,-0.405295
1,862909,2630,646,107863,shopping,1,False,1,Home,1005,1122,NaN,15.000000,SHARED3FREE,-1.118461
2,1493697,4553,2580,186712,work,1,True,1,work,435,998,NaN,8.000000,DRIVEALONEPAY,-1.508845
3,1493701,4553,2580,186712,work,1,False,1,Home,998,435,NaN,18.000000,SHARED3FREE,-1.980250
4,1494025,4554,2580,186753,work,1,True,1,work,1152,998,NaN,7.000000,SHARED3FREE,0.161330


In [ ]:
final_trips_2010_3_noPlan = final_trips_2010_3_noPlan.sort_values(by=['person_id', 'tour_id']).reset_index(drop=True)

In [ ]:
Person_Trip_eventsSF

#### New

In [108]:
# Column with five summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car']

In [109]:
Person_Trip_eventsSF['mode_choice_actual_5'] = np.select(conditions, choices, default= np.nan)

In [110]:
# Column with six summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car', 'ride_hail_transit']

In [111]:
Person_Trip_eventsSF['mode_choice_actual_6'] = pd.Series(np.select(conditions, choices, default= np.nan)).replace({'nan':np.nan})

In [112]:
# Column with four summarized modes
Person_Trip_eventsSF['mode_choice_actual_4']  = np.where((Person_Trip_eventsSF['mode_choice_actual_5'] == 'walk')|(Person_Trip_eventsSF['mode_choice_actual_5'] == 'bike'),
                                                        'walk/bike', Person_Trip_eventsSF['mode_choice_actual_5'])

In [113]:
Person_Trip_eventsSF = Person_Trip_eventsSF.drop(Person_Trip_eventsSF[Person_Trip_eventsSF.duration_door_to_door < 0].index)

In [141]:
#Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_base_core_act.csv', index = False)

#### Merging with ActivitySim files

In [114]:
households = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/activitysim/year-2010-iteration-3/households.csv.gz', compression = 'gzip', dtype = dtypes)

In [115]:
persons = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/activitysim/year-2010-iteration-3/persons.csv.gz', compression = 'gzip', dtype = dtypes)

In [116]:
tours = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/activitysim/year-2010-iteration-3/final_tours.csv.gz', compression = 'gzip', dtype = dtypes)

In [117]:
trips = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/activitysim/year-2010-iteration-3/final_trips.csv.gz', compression = 'gzip', dtype = dtypes)

In [45]:
plans = pd.read_csv('gs://beam-core-outputs/sfbay-2010-base-noplanmemory-20221205/activitysim/year-2010-iteration-3/plans.csv.gz', compression = 'gzip', dtype = dtypes)

In [53]:
tours[tours['person_id']==115]

,tour_id,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,tdd,start,end,duration,composition,destination_logsum,tour_mode,mode_choice_logsum,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
205176,4746,115,school,1,1,1,1,mandatory,1,626.000000,1005.000000,1114,43.000000,7.000000,13.000000,6.000000,NaN,NaN,SHARED2FREE,-2.648813,NaN,NaN,0out_0in,school


In [40]:
eventsSF[eventsSF['tripId'] == 37970]

,person,reason,time,type,vehicle,actType,vehicleType,price,fuel,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,duration,currentTourMode,endY,endX,startY,startX,arrivalTime,departureTime,link,facility,departTime,requireWheelchair,mode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,driver,score,primaryFuelLevel,secondaryFuelLevel,cost,links,numPassengers,primaryFuel,secondaryFuelType,primaryFuelType,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,incentive,tollCost,netCost,legMode
5921262,115,NaN,29812.000000,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hov2_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hov2_teleportation,NaN,HOV2_TELEPORTATION,172346.000000,true,15006.400000,1.000000,"WALK,CAR_HOV2,WALK","body-115,teleportationSharedVehicle-2-115,body...",escort,school,37970.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
trips[trips['trip_id'] == 37970]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum


In [118]:
# Merge BEAM households and persons 
persons = persons.sort_values(by=['household_id']).reset_index(drop=True)
households = households.sort_values(by=['household_id']).reset_index(drop=True)
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [119]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['person_id']).reset_index(drop=True)
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [120]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id']).reset_index(drop=True)
hhperTours = hhperTours.sort_values(by=['person_id','tour_id']).reset_index(drop=True)
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [129]:
# Concat mode_choice_utilities files
path = "C:/Users/nazanin/Downloads/sfbay-2010-base-noplanmemory-20221205_activitysim_year-2010-iteration-3_trip_mode_choice/trip_mode_choice/" #the path should be updated
all_files = glob.glob(path + "*utilities.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_utilities = pd.concat(li2, axis = 0, ignore_index=True)

In [147]:
#just utilities
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=tourTripsMerged, right=SFmode_choice_utilities, how='left', on=['trip_id'])

In [145]:
SFmode_choice_utilities[SFmode_choice_utilities['trip_id']==1518]

,trip_id,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
465382,1518,-0.739763,-0.745372,-999.690602,-999.693808,-999.670938,-999.673182,-20.235781,-1001.427336,-1998.626158,-1998.356056,-1998.635056,-1998.356056,-1998.356056,-1998.402928,-1998.402928,-1998.402928,-1998.402928,-1998.402928,-7.668343,-5.374940,-3.687102


In [149]:
SFActMerged[SFActMerged['trip_id']==1518]

,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode,mode_choice_logsum_x,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode,mode_choice_logsum_y,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
2,1518,4,865,189,othdiscr,2,False,2,Home,1005,977,NaN,16.000000,DRIVEALONEFREE,-0.524496,othdiscr,1,1,1,1,non_mandatory,1,401.000000,1005.000000,865,146.000000,15.000000,16.000000,1.000000,NaN,12.570320,DRIVEALONEFREE,-2.177580,NaN,NaN,0out_1in,othdiscr,68,0.000000,22.000000,0,0.000000,1,2,0,2,0,0,0,865,above 60,female,black,0,no,1,1005,5,3,3,-122.227937,37.854097,False,True,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0,False,1005,2.818730,-1,NaN,0.000000,-1,NaN,NaN,False,NaN,0.000000,0.000000,0.000000,False,N,True,False,False,False,NaN,False,False,0,0,0,1,1,0,0,0,0,1,0,1,2012000224482,3.000000,70550.000000,2,68,0.000000,0,1,0,yes,gt65,black,no,two,two or more,no,2.000000,gt60-lt100,none,own not recent,yes,60014001001061,6001,2,1,1,4,1005,4,0.105000,73380,70.550000,3,10.440000,2.818730,2.000000,2,2,0,0,0,0,0,0,True,False,False,False,1,0.000000,0,2,2,0,0,2,True,0_tours,0,-0.739763,-0.745372,-999.690602,-999.693808,-999.670938,-999.673182,-20.235781,-1001.427336,-1998.626158,-1998.356056,-1998.635056,-1998.356056,-1998.356056,-1998.402928,-1998.402928,-1998.402928,-1998.402928,-1998.402928,-7.668343,-5.374940,-3.687102


In [150]:
Person_Trip_eventsSF[Person_Trip_eventsSF['tripIndex']==1518]

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4
2,4,1518.000000,67199.000000,social,68335.000000,Home,0.679361,0.000000,7352.261000,6194.061000,0.000000,0.000000,7352.261000,1158.200000,245.000000,0.000000,0.000000,0.000000,1136.000000,891.000000,0.000000,0.000000,0.000000,0.000000,0.000948,0.000000,0.000000,0.000000,0.000000,61384.600000,12822557.735979,12883942.335979,12822557.735979,"body-4,358558,358558,body-4",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Residential",,0,0,baseline,0,0,0,0,0,"358558, body-4",2010,1136.000000,0.000000,social_to_Home,car,car,car


In [154]:
eventsASim[eventsASim['tripIndex']==1518.000000]

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode,mode_choice_logsum_x,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode,mode_choice_logsum_y,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
2,4,1518.000000,67199.000000,social,68335.000000,Home,0.679361,0.000000,7352.261000,6194.061000,0.000000,0.000000,7352.261000,1158.200000,245.000000,0.000000,0.000000,0.000000,1136.000000,891.000000,0.000000,0.000000,0.000000,0.000000,0.000948,0.000000,0.000000,0.000000,0.000000,61384.600000,12822557.735979,12883942.335979,12822557.735979,"body-4,358558,358558,body-4",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Residential",,0,0,baseline,0,0,0,0,0,"358558, body-4"

In [130]:
# Both raw and utilities
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
#tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
#rawUtil = rawUtil.sort_values(by=['trip_id'])
#SFActMerged= pd.merge(left=tourTripsMerged, right=rawUtil, how='left', on=['trip_id'])

In [151]:
# Merge person_trip level BEAM with activity sim merged files
SFActMerged = SFActMerged.sort_values(by=['person_id', 'trip_id']).reset_index(drop=True)
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex']).reset_index(drop=True)
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=SFActMerged, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

#### Updated the cells below - Might consider adding the INEXUS metrics as well

In [155]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [156]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [157]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_Potential_INEXUS"}, inplace=True)

In [158]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [159]:
# Add a column of income quartiles
quartiles = eventsASim['income'].quantile([0,.25, .5, .75,1]).tolist()

In [160]:
# Add income deciles
conditions  = [(eventsASim['income'] >= quartiles[0]) & (eventsASim['income'] < quartiles[1]), 
               (eventsASim['income'] >= quartiles[1]) & (eventsASim['income'] < quartiles[2]),
               (eventsASim['income'] >=  quartiles[2]) & (eventsASim['income'] < quartiles[3]),
               (eventsASim['income'] >= quartiles[3]) & (eventsASim['income'] <= quartiles[4])]

choices = [ '1stQ', '2ndQ', '3rdQ', '4thD']

In [161]:
eventsASim['income_quartiles'] = np.select(conditions, choices, default=None)

In [162]:
# Add a column of income deciles
deciles = eventsASim['income'].quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).tolist()

In [163]:
# Add income deciles
conditions  = [(eventsASim['income'] >= deciles[0]) & (eventsASim['income'] < deciles[1]), 
               (eventsASim['income'] >= deciles[1]) & (eventsASim['income'] < deciles[2]),
               (eventsASim['income'] >=  deciles[2]) & (eventsASim['income'] < deciles[3]),
               (eventsASim['income'] >= deciles[3]) & (eventsASim['income'] < deciles[4]), 
               (eventsASim['income'] >=  deciles[4]) & (eventsASim['income'] < deciles[5]),
               (eventsASim['income'] >=  deciles[5]) & (eventsASim['income'] < deciles[6]),
               (eventsASim['income'] >=  deciles[6]) & (eventsASim['income'] < deciles[7]),
               (eventsASim['income'] >=  deciles[7]) & (eventsASim['income'] < deciles[8]),
               (eventsASim['income'] >=  deciles[8]) & (eventsASim['income'] < deciles[9]),
               (eventsASim['income'] >=  deciles[9]) & (eventsASim['income'] <= deciles[10])]

choices = [ '1stD', '2ndD', '3rdD', 
           '4thD', '5thD', '6thD', '7thD', '8thD', '9thD','10thD']

In [164]:
eventsASim['income_deciles'] = np.select(conditions, choices, default=None)

In [165]:
eventsASim[eventsASim['logsum_tours_mode_AS_tours'].isna()].shape

(0, 256)

In [204]:
# Save the output file to S3
eventsASim.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Baseline/sf_2015_baseline_test.csv', index=False)  #the path should be updated

#### INEXUS

In [166]:
# Mapping BEAM and ASIM modes
conditions  = [(eventsASim['mode_choice_actual_BEAM'] == 'walk')&(eventsASim['trip_mode_AS_trips'] == 'WALK'),
               (eventsASim['mode_choice_actual_BEAM'] == 'bike')&(eventsASim['trip_mode_AS_trips'] == 'BIKE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(eventsASim['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(eventsASim['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (eventsASim['mode_choice_actual_BEAM'] == 'car_hov3')&(eventsASim['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'car_hov3')&(eventsASim['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (eventsASim['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(eventsASim['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(eventsASim['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (eventsASim['mode_choice_actual_BEAM'] == 'car_hov2')&(eventsASim['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (eventsASim['mode_choice_actual_BEAM'] == 'car_hov2')&(eventsASim['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'car')&(eventsASim['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'car')&(eventsASim['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (eventsASim['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim['trip_mode_AS_trips'] == 'WALK_LRF'),
               (eventsASim['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim['trip_mode_AS_trips'] == 'WALK_LOC'),
               (eventsASim['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim['trip_mode_AS_trips'] == 'WALK_EXP'),
               (eventsASim['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim['trip_mode_AS_trips'] == 'WALK_HVY'),
               (eventsASim['mode_choice_actual_BEAM'] == 'ride_hail')&(eventsASim['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (eventsASim['mode_choice_actual_BEAM'] == 'ride_hail')&(eventsASim['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (eventsASim['mode_choice_actual_BEAM'] == 'ride_hail')&(eventsASim['trip_mode_AS_trips'] == 'TAXI'),
               (eventsASim['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim['trip_mode_AS_trips'] == 'WALK_COM'),
               (eventsASim['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (eventsASim['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (eventsASim['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (eventsASim['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (eventsASim['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim['trip_mode_AS_trips'] == 'DRIVE_HVY')]


choices = [eventsASim['WALK'], eventsASim['BIKE'], eventsASim['SHARED3FREE'], eventsASim['SHARED3PAY'],
           eventsASim['SHARED3FREE'], eventsASim['SHARED3PAY'], eventsASim['SHARED2FREE'], 
           eventsASim['SHARED2PAY'], eventsASim['SHARED2PAY'], eventsASim['SHARED2FREE'], 
           eventsASim['DRIVEALONEFREE'], eventsASim['DRIVEALONEPAY'], eventsASim['WALK_LRF'], 
           eventsASim['WALK_LOC'], eventsASim['WALK_EXP'], eventsASim['WALK_HVY'], eventsASim['TNC_SINGLE'], 
           eventsASim['TNC_SHARED'], eventsASim['TAXI'], eventsASim['WALK_COM'], 
           eventsASim['DRIVE_COM'], eventsASim['DRIVE_LRF'], eventsASim['DRIVE_LOC'], eventsASim['DRIVE_EXP'],
           eventsASim['DRIVE_HVY']]

In [167]:
eventsASim['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)

In [173]:
eventsASim['Realized_INEXUS'].max()

2.4034145008291734

In [177]:
eventsASim.head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_Potential_INEXUS,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,income_quartiles,income_deciles,Realized_INEXUS
0,4,1513.000000,57513.000000,Home,61414.000000,othdiscr,6.609155,0.000000,60258.855000,62966.636000,0.000000,0.000000,64733.977000,1767.341000,2542.000000,0.000000,0.000000,0.000000,3901.000000,1359.000000,0.000000,0.000000,0.000000,0.000000,0.007940,0.000000,0.000000,0.000000,0.000000,93669.073000,107418015.519458,107511684.592458,107418015.519458,"body-4,358558,358558,body-4",default,1,c

In [178]:
eventsASim.replanning_status.unique()

array([0, 1, 3, 2])

In [179]:
eventsASim[eventsASim['replanning_status'] == 1].head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_Potential_INEXUS,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,income_quartiles,income_deciles,Realized_INEXUS
69,169,55637.000000,78973.000000,othdiscr,81202.000000,Home,13.988558,0.000000,43158.272000,0.000000,28005.781000,0.000000,28005.781000,0.000000,0.000000,1252.000000,0.000000,0.000000,1252.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003780,0.000126,0.000000,0.000000,0.000000,0.000000,51137602.818860,51137602.818860,51137602.818860,"body-169,rideHailVehicle-3222022@GlobalRHM,bod..

In [181]:
eventsASim[eventsASim['Realized_INEXUS'].isna()].head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_Potential_INEXUS,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,student,work_at_home,worker,household_id_y,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,serialno,VEHICL,income,race_of_head,age_of_head,num_workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,hh_age_of_head,hh_race_of_head,hispanic_head,hh_size,hh_cars,hh_children,seniors,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,hhsize,gt55,gt2,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,income_quartiles,income_deciles,Realized_INEXUS
25,125,41317.000000,67013.000000,work,68599.000000,Home,4.798850,0.000000,43820.254000,43753.428000,0.000000,0.000000,43820.254000,66.826000,1535.000000,0.000000,0.000000,0.000000,1586.000000,51.000000,0.000000,0.000000,0.000000,0.000000,0.005468,0.000000,0.000000,0.000000,0.000000,3541.778000,73980273.838415,73983815.616415,73980273.838415,"body-125,51823,51823,body-125",default,1,car,car

In [200]:
eventsSF[eventsSF['person'] == '165']

,IDMerged,person,driver,riders,link,legMode,time,type,primaryFuelLevel,secondaryFuelLevel,price,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,vehicle,currentTourMode,endY,endX,startY,startX,arrivalTime,departureTime,reason,facility,actType,vehicleType,shiftStatus,parkingZoneId,fuel,duration,score,modeBEAM,incentive,tollCost,cost_BEAM,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,departTime,requireWheelchair,links,numPassengers,primaryFuel,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,toStopIndex,fromStopIndex,cost,scenario,lever,year,lever_position,modeBEAM_rh,actEndTime,actStartTime,duration_travelling,distance_travelling,distance_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,fuelFood,emissionFood,fuelElectricity,emissionElectricity,fuelDiesel,emissionDiesel,fuelBiodiesel,emissionBiodiesel,fuel_not_Food,fuelGasoline,emissionGasoline,fuel_marginal,emission_marginal,actEndType,actStartType,tripIndex,mode_choice_actual_BEAM,mode_choice_planned_BEAM,replanning_status,transit_bus,transit_subway,transit_tram,transit_rail,transit_cable_car,ride_hail_pooled
587,165,165,NaN,NaN,NaN,NaN,29574.000000,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,NaN,NaN,NaN,NaN,CAR,62008.000000,true,6748.330000,1.000000,"WALK,CAR,CAR,WALK","body-165,718534,718534,body-165",Home,othmaint,54433.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,default,2010,1,car,NaN,NaN,NaN,NaN,6748.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,54433.000000,car,car,0,0,0,0,0,0,0
588,165,165,NaN,NaN,62008.000000,NaN,29574.000000,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,default,2010,1,NaN,29574.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,,54433.000000,NaN,NaN,0,0,0,0,0,0,0
589,165,165,NaN,NaN,62008.000000,car,29574.000000,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,default,2010,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,54433.000000,NaN,NaN,0,0,0,0,0,0,0
590,165,165,NaN,NaN,NaN,NaN,29574.000000,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,body-165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,default,2010,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,54433.000000,NaN,NaN,0,0,0,0,0,0,0
592,165,165,NaN,NaN,NaN,NaN,29574.000000,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,718534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,default,2010,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,54433.000000,NaN,NaN,0,0,0,0,0,0,0
597,165,165,NaN,NaN,NaN,NaN,29973.000000,PersonCost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [201]:
Person_Trip_eventsSF[Person_Trip_eventsSF['IDMerged']==165].sort_values(by ='actEndTime')

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,lever,lever_position,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4
45,165,54433.000000,29574.000000,Home,29973.000000,othmaint,0.740153,0.000000,6748.330000,6748.330000,0.000000,0.000000,6748.330000,0.000000,399.000000,0.000000,0.000000,0.000000,399.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000743,0.000000,0.000000,0.000000,0.000000,0.000000,10055385.378836,10055385.378836,10055385.378836,"body-165,718534,718534,body-165",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Residential, Public",,0,0,baseline,0,0,0,0,0,"718534, body-165",2010,399.000000,0.000000,Home_to_othmaint,car,car,car
46,165,54434.000000,30801.000000,othmaint,31332.000000,work,0.778562,0.000000,7239.868000,7098.526000,0.000000,0.000000,7239.868000,141.342000,422.000000,0.000000,0.000000,0.000000,531.000000,109.000000,0.000000,0.000000,0.000000,0.000000,0.000824,0.000000,0.000000,0.000000,0.000000,7491.126000,11149694.524115,11157185.650115,11149694.524115,"body-165,718534,718534,body-165",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Workplace",,0,0,baseline,0,0,0,0,0,"718534, body-165",2010,531.000000,0.000000,othmaint_to_work,car,car,car
37,165,54305.000000,38203.000000,work,40865.000000,atwork,6.403225,0.000000,58861.152000,58381.289000,0.000000,0.000000,58861.152000,479.863000,2293.000000,0.000000,0.000000,0.000000,2662.000000,369.000000,0.000000,0.000000,0.000000,0.000000,0.006897,0.000000,0.000000,0.000000,0.000000,25432.739000,93309962.840181,93335395.579181,93309962.840181,"body-165,718534,718534,body-165",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Workplace, Workplace",,0,0,baseline,0,0,0,0,0,"718534, body-165",2010,2662.000000,0.000000,work_to_atwork,car,car,car
47,165,54437.000000,57681.000000,atwork,60364.000000,shopping,1.038297,0.000000,9831.363000,10649.094000,0.000000,0.000000,13538.854000,2889.760000,460.000000,0.000000,0.000000,0.000000,2683.000000,2223.000000,0.000000,0.000000,0.000000,0.000000,0.001267,0.000000,0.000000,0.000000,0.000000,153157.280000,17145749.501569,17298906.781569,17145749.501569,"body-165,718534,718534,body-165",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Workplace, Workplace, Public",,0,0,baseline,0,0,0,0,0,"718534, body-165",2010,2683.000000,0.000000,atwork_to_shopping,car,car,car
38,165,54309.000000,60514.000000,shopping,62705.000000,work,1.041527,0.000000,11666.901000,9496.104000,0.000000,0.000000,11666.901000,2170.797000,521.000000,0.000000,0.000000,0.000000,2191.000000,1670.000000,0.000000,0.000000,0.000000,0.000000,0.001038,0.000000,0.000000,0.000000,0.000000,115052.241000,14043057.971399,14158110.212399,14043057.971399,"body-165,718534,718534,body-165",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, Public",,0,0,baseline,0,0,0,0,0,"718534, body-165",2010,2191.000000,0.000000,shopping_to_work,car,car,car
39,165,54310.000000,62855.000000,work,64004.000000,work,2.100048,0.000000,19459.682000,19147.155000,0.000000,0.000000,19459.682000,312.527000,909.000000,0.000000,0.000000,0.000000,1149.000000,240.000000,0.000000,0.000000,0.000000,0.000000,0.002747,0.000000,0.000000,0.000000,0.000000,16563.931000,37171655.620404,37188219.551404,37171655.620404,"body-165,718534,718534,body-165",default,1,car,car,"1.0, 0.0, 0.0, 1.0","Public, 

In [203]:
eventsASim_temp[eventsASim_temp['IDMerged']== 175].sort_values(by ='actEndTime')

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,mode_choice_planned_BEAM,mode_choice_actual_BEAM,trip_mode_AS_trips,tour_mode_AS_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
89,175,57713.000000,23353.000000,Home,62173.000000,work,walk,walk,SHARED3PAY,SHARED3FREE,-0.011524,-0.011731,-1.207124,-1.207242,-0.195124,-0.195207,-171.319869,-1076.243327,-1998.634164,-1998.634164,-1998.634164,-2001.300070,-2001.682397,-1002.796853,-1998.341124,-1998.341124,-2002.230783,-2002.949247,-72.910172,-74.446088,-73.622383,NaN
90,175,57717.000000,62323.000000,work,97810.000000,eatout,walk,walk,SHARED3PAY,SHARED3FREE,-2.093193,-2.093398,-3.285840,-3.285957,-2.272658,-2.272740,-157.234634,-1069.200709,-1998.634164,-1998.634164,-1998.634164,-1998.634164,-1998.634164,-1998.341124,-1998.341124,-1998.341124,-1998.341124,-1998.341124,-98.722925,-87.740191,-79.343618,NaN
91,175,57718.000000,97960.000000,eatout,102475.000000,Home,walk,walk,SHARED3PAY,SHARED3FREE,-0.638634,-0.639472,-1.831280,-1.831759,-0.818099,-0.818434,-13.783926,-1001.153347,-1999.185960,-1998.628085,-1998.848085,-1998.628085,-1998.628085,-1998.335045,-1998.335045,-1998.335045,-1998.335045,-1998.335045,-78.083341,-76.665777,-75.119378,NaN


In [194]:
eventsASim_temp[eventsASim_temp['Realized_INEXUS'].isna()].head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,mode_choice_planned_BEAM,mode_choice_actual_BEAM,trip_mode_AS_trips,tour_mode_AS_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
37,165,54305.000000,38203.000000,work,40865.000000,atwork,car,car,WALK,WALK,-999.855789,-999.856698,-999.855789,-999.856308,-999.855789,-999.856153,-7.783142,-1999.218253,-1003.005291,-1998.143311,-1998.143311,-1998.143311,-1998.143311,-999.949953,-1998.311395,-1998.311395,-1998.311395,-1998.311395,-9.410813,-9.005360,-9.426347,NaN
39,165,54310.000000,62855.000000,work,64004.000000,work,car,car,WALK,WALK,-999.692398,-999.693115,-999.685031,-999.685441,-999.682085,-999.682371,-7.783142,-1999.218253,-1001.171600,-1998.143311,-1998.422311,-1998.143311,-1998.143311,-1998.311395,-1998.311395,-1998.311395,-1998.311395,-1998.311395,-8.991587,-8.525298,-9.060521,NaN
89,175,57713.000000,23353.000000,Home,62173.000000,work,walk,walk,SHARED3PAY,SHARED3FREE,-0.011524,-0.011731,-1.207124,-1.207242,-0.195124,-0.195207,-171.319869,-1076.243327,-1998.634164,-1998.634164,-1998.634164,-2001.300070,-2001.682397,-1002.796853,-1998.341124,-1998.341124,-2002.230783,-2002.949247,-72.910172,-74.446088,-73.622383,NaN
90,175,57717.000000,62323.000000,work,97810.000000,eatout,walk,walk,SHARED3PAY,SHARED3FREE,-2.093193,-2.093398,-3.285840,-3.285957,-2.272658,-2.272740,-157.234634,-1069.200709,-1998.634164,-1998.634164,-1998.634164,-1998.634164,-1998.634164,-1998.341124,-1998.341124,-1998.341124,-1998.341124,-1998.341124,-98.722925,-87.740191,-79.343618,NaN
91,175,57718.000000,97960.000000,eatout,102475.000000,Home,walk,walk,SHARED3PAY,SHARED3FREE,-0.638634,-0.639472,-1.831280,-1.831759,-0.818099,-0.818434,-13.783926,-1001.153347,-1999.185960,-1998.628085,-1998.848085,-1998.628085,-1998.628085,-1998.335045,-1998.335045,-1998.335045,-1998.335045,-1998.335045,-78.083341,-76.665777,-75.119378,NaN


In [182]:
eventsASim[eventsASim['Realized_INEXUS'].isna()].shape

(540462, 257)

In [184]:
eventsASim_temp = eventsASim[['IDMerged', 'tripIndex','actEndTime','actEndType','actStartTime','actStartType',
                                   'mode_choice_planned_BEAM','mode_choice_actual_BEAM','trip_mode_AS_trips','tour_mode_AS_tours',
                                  'DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED2PAY','SHARED3FREE','SHARED3PAY','WALK',
                                   'BIKE','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF',
                                   'DRIVE_EXP','DRIVE_HVY','DRIVE_COM','TAXI','TNC_SINGLE','TNC_SHARED']]

In [188]:
# Mapping BEAM and ASIM modes
conditions  = [(eventsASim_temp['mode_choice_actual_BEAM'] == 'walk')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'bike')&(eventsASim_temp['trip_mode_AS_trips'] == 'BIKE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov3')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov3')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov2')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov2')&(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_LRF'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_LOC'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_EXP'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_HVY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_COM'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail')&(eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail')&(eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail')&(eventsASim_temp['trip_mode_AS_trips'] == 'TAXI'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_HVY'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(eventsASim_temp['trip_mode_AS_trips'] == 'TAXI'),
       
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov2_teleportation')&((eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3FREE')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3PAY')|(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEPAY')),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov2')&((eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3PAY')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3FREE')|(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEPAY')),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov3')&((eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2PAY')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2FREE')|(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEPAY')),                      
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov3_teleportation')&((eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEPAY')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2PAY')|(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2FREE')),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car')&((eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3FREE')|(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2FREE')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'SHARED3PAY')|(eventsASim_temp['trip_mode_AS_trips'] == 'SHARED2PAY')|(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_LOC')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_LRF')|(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_EXP')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_HVY')|(eventsASim_temp['trip_mode_AS_trips'] == 'DRIVE_COM')),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk')&((eventsASim_temp['trip_mode_AS_trips'] == 'WALK_LRF')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'WALK_LOC')|(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_EXP')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'WALK_HVY')|(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_COM')),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(eventsASim_temp['trip_mode_AS_trips'] == 'WALK'),
               ((eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail_pooled')|(eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail'))
               &(~((eventsASim_temp['trip_mode_AS_trips'] == 'TAXI')|(eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SHARED')|
                (eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SINGLE'))),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit')&(~((eventsASim_temp['trip_mode_AS_trips'] == 'WALK_LRF')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'WALK_LOC')|(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_EXP')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'WALK_HVY')|(eventsASim_temp['trip_mode_AS_trips'] == 'WALK_COM')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'WALK'))),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'bike_transit'),
               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car')&((eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SHARED')|
               (eventsASim_temp['trip_mode_AS_trips'] == 'TNC_SINGLE')|(eventsASim_temp['trip_mode_AS_trips'] == 'TAXI'))]
               #(eventsASim_temp['mode_choice_actual_BEAM'] == 'ride_hail_transit')
                                                                      
                                                                                                                 
                                                                      
choices = [eventsASim_temp['WALK'], eventsASim_temp['BIKE'], eventsASim_temp['SHARED3FREE'], eventsASim_temp['SHARED3PAY'],
           eventsASim_temp['SHARED3FREE'], eventsASim_temp['SHARED3PAY'], eventsASim_temp['SHARED2FREE'], 
           eventsASim_temp['SHARED2PAY'], eventsASim_temp['SHARED2PAY'], eventsASim_temp['SHARED2FREE'], 
           eventsASim_temp['DRIVEALONEFREE'], eventsASim_temp['DRIVEALONEPAY'], eventsASim_temp['WALK_LRF'], 
           eventsASim_temp['WALK_LOC'], eventsASim_temp['WALK_EXP'], eventsASim_temp['WALK_HVY'], eventsASim_temp['WALK_COM'],
           eventsASim_temp['TNC_SINGLE'], 
           eventsASim_temp['TNC_SHARED'], eventsASim_temp['TAXI'],
           eventsASim_temp['DRIVE_COM'], eventsASim_temp['DRIVE_LRF'], eventsASim_temp['DRIVE_LOC'], eventsASim_temp['DRIVE_EXP'],
           eventsASim_temp['DRIVE_HVY'], eventsASim_temp['TNC_SINGLE'], eventsASim_temp['TNC_SHARED'], eventsASim_temp['TAXI'],
          eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM']].max(axis=1),
          eventsASim_temp[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM']].max(axis=1),
          eventsASim_temp[['TAXI','TNC_SINGLE','TNC_SHARED']].max(axis=1),
          eventsASim_temp[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['BIKE','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1)]
          #eventsASim_temp[['TAXI','TNC_SHARED','TNC_SINGLE','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1)]

In [189]:
eventsASim_temp['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_868\3927896634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventsASim_temp['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)


In [190]:
eventsASim_temp[eventsASim_temp['Realized_INEXUS'].isna()].shape

(94895, 32)

In [191]:
eventsASim_temp[eventsASim_temp['Realized_INEXUS']<-1000].shape

(499, 32)

In [398]:
eventsASim_temp['Realized_INEXUS'] = np.where(((eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov2')|(eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov3')|
                                               (eventsASim_temp['mode_choice_actual_BEAM'] == 'car')|(eventsASim_temp['mode_choice_actual_BEAM'] == 'hov2_teleportation')|
                                               (eventsASim_temp['mode_choice_actual_BEAM'] == 'hov3_teleportation')) & ((eventsASim_temp['Realized_INEXUS'].isna())), 
                                              eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1), eventsASim_temp['Realized_INEXUS'])

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_3592\4117822543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventsASim_temp['Realized_INEXUS'] = np.where(((eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov2')|(eventsASim_temp['mode_choice_actual_BEAM'] == 'car_hov3')|


In [405]:
eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'walk') & (eventsASim_temp['Realized_INEXUS'].isna()), 
                                              eventsASim_temp[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM']].max(axis=1), eventsASim_temp['Realized_INEXUS'])

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_3592\2836354814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'walk') & (eventsASim_temp['Realized_INEXUS'].isna()),


In [406]:
eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit') & (eventsASim_temp['Realized_INEXUS'].isna()), 
                                              eventsASim_temp[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1), eventsASim_temp['Realized_INEXUS'])

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_3592\1416759140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'walk_transit') & (eventsASim_temp['Realized_INEXUS'].isna()),


In [414]:
eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit') & (eventsASim_temp['Realized_INEXUS'].isna()), 
                                              eventsASim_temp[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1), eventsASim_temp['Realized_INEXUS'])

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_3592\2314472807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'drive_transit') & (eventsASim_temp['Realized_INEXUS'].isna()),


In [420]:
eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'bike') & (eventsASim_temp['Realized_INEXUS'].isna()), 
                                              eventsASim_temp[['BIKE']].max(axis=1), eventsASim_temp['Realized_INEXUS'])

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_3592\624428316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventsASim_temp['Realized_INEXUS'] = np.where((eventsASim_temp['mode_choice_actual_BEAM'] == 'bike') & (eventsASim_temp['Realized_INEXUS'].isna()),


In [421]:
eventsASim_temp[(eventsASim_temp['Realized_INEXUS'] < -900)].shape

(78164, 31)

In [422]:
eventsASim_temp[(eventsASim_temp['Realized_INEXUS'].isna())].shape

(0, 31)

In [423]:
eventsASim_temp[(eventsASim_temp['Realized_INEXUS'] < -900)].head(10)

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,mode_choice_planned_BEAM,mode_choice_actual_BEAM,trip_mode_AS_trips,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,Realized_INEXUS
10,113,37377.000000,24680.000000,Home,27699.000000,work,bike,bike,WALK_LOC,-999.468219,-999.471688,-4.131319,-4.133301,-4.599619,-4.601007,-10.446809,-1000.794649,-0.756909,-999.113100,-999.930600,-999.221400,-999.174000,-999.842644,-1998.282160,-1998.282160,-1999.051699,-1998.282160,-4.809695,-4.856696,-5.389089,-1000.794649
11,113,37381.000000,64328.000000,work,65091.000000,Home,bike,bike,WALK_LOC,-999.628319,-999.628817,-4.161177,-4.161461,-4.577380,-4.577579,-13.078193,-1001.057787,-1.673366,-999.113100,-999.930600,-999.221400,-999.174000,-1998.282160,-1998.282160,-1998.282160,-1998.282160,-1998.282160,-5.026477,-4.932365,-5.097281,-1001.057787
94,187,61649.000000,22449.000000,Home,23259.000000,work,bike,bike,DRIVEALONEFREE,-0.429053,-0.431843,-999.429053,-999.430648,-999.429053,-999.430169,-6.954524,-1000.348259,-999.906916,-1998.609799,-1998.609799,-1998.609799,-1998.609799,-1000.104329,-1998.316759,-1998.316759,-1999.089515,-1998.316759,-9.134040,-8.882413,-8.913232,-1000.348259
95,187,61653.000000,57211.000000,work,58515.000000,Home,bike,bike,DRIVEALONEPAY,-0.493458,-0.494159,-999.433171,-999.433572,-999.409057,-999.409337,-6.099205,-1000.262728,-1998.619054,-1998.609799,-1998.609799,-1998.609799,-1998.609799,-1998.316759,-1998.316759,-1998.316759,-1998.316759,-1998.316759,-8.683719,-8.477578,-8.704530,-1000.262728
119,257,84329.000000,36194.000000,work,37316.000000,atwork,car,car,WALK,-1998.231232,-1998.231379,-999.231232,-999.231316,-999.231232,-999.231291,-0.393102,-1998.313522,-999.388514,-1998.098654,-1998.098654,-1998.098654,-1998.098654,-1998.266738,-1998.266738,-1998.266738,-1998.266738,-1998.266738,-8.131405,-8.237605,-8.361245,-999.231232
120,257,84333.000000,37466.000000,atwork,37641.000000,work,car,car,WALK,-1998.231232,-1998.231379,-999.231232,-999.231316,-999.231232,-999.231291,-0.393102,-1998.313522,-999.388514,-1998.098654,-1998.098654,-1998.098654,-1998.098654,-1998.266738,-1998.266738,-1998.266738,-1998.266738,-1998.266738,-8.131405,-8.237605,-8.361245,-999.231232
323,700,229633.000000,37346.000000,work,39662.000000,atwork,car,car,WALK,-999.088188,-999.088283,-999.088188,-999.088242,-999.088188,-999.088226,-0.341314,-1998.210950,-999.766602,-1997.945298,-1997.945298,-1997.945298,-1997.945298,-1998.222061,-1998.222061,-1998.222061,-1998.222061,-1998.222061,-7.795621,-7.513685,-7.804696,-999.088188
324,700,229637.000000,47563.000000,atwork,48377.000000,work,car,car,WALK,-999.896211,-999.896406,-999.617344,-999.617456,-999.505798,-999.505876,-0.403872,-1998.242229,-1000.241696,-1997.945298,-1997.945298,-1997.945298,-1997.945298,-1998.222061,-1998.222061,-1998.222061,-1998.222061,-1998.222061,-8.708262,-8.297894,-8.131228,-999.505798
394,856,280969.000000,131800.000000,Home,132517.000000,othdiscr,hov3_teleportation,hov3_teleportation,BIKE,-1000.422650,-1000.514278,-1000.422650,-1000.475009,-1000.422650,-1000.459302,-27.429583,-3.145419,-1002.437409,-1998.333171,-1998.333171,-1999.616409,-1998.333171,-999.806252,-1998.380043,-1998.380043,-1998.380043,-1998.380043,-22.579017,-18.741263,-13.346698,-999.806252
395,856,280973.000000,132667.000000,othdiscr,133487.000000,Home,hov3_teleportation,hov3_teleportation,BIKE,-1001.423043,-1001.470952,-1000.791505,-1000.818881,-1000.538889,-1000.558053,-25.284398,-2.930901,-1998.911885,-1998.333171,-1998.333171,-1998.333171,-1998.333171,-1998.380043,-1998.380043,-1998.380043,-1998.380043,-1998.380043,-17.699244,-15.262166,-12.138130,-1000.538889


In [419]:
eventsASim_temp[eventsASim_temp['Realized_INEXUS'].isna()].to_csv("C:/Shared-Work/Data/CleanData/sample_test.csv", index =False)

In [418]:
eventsASim_temp.head(10000).to_csv("C:/Shared-Work/Data/CleanData/sample_test_2.csv", index =False)

In [122]:
# Delete the utilities files downloaded and saved in the system
os.remove('C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice.zip')   #the path should be updated
shutil.rmtree('C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice')   #the path should be updated